## Hate Speech Detection with model

In [2]:
import pandas as pd
train = pd.read_csv(r"C:\Users\SALOME\Downloads\Requirments (10)\train.csv")
print("Training set:" %train.columns, train.shape, len(train))
test = pd.read_csv(r"C:\Users\SALOME\Downloads\Requirments (10)\test.csv")
print("Test set:" %test.columns, test.shape, len(test))

Training set: (31962, 3) 31962
Test set: (17197, 2) 17197


In [3]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [8]:
import re
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)^rt|https.+?", "", elem))
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

In [9]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label == 0]
train_minority = train_clean[train_clean.label ==1]
train_minority_upsampled = resample(train_minority,
                                    replace= True,
                                    n_samples = len(train_majority),
                                    random_state = 123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

label
1    29720
0    29720
Name: count, dtype: int64

In [13]:
# Scikit-learn's Pipeline with an SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn .feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SGDClassifier()),
])

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_upsampled["tweet"],
                                                    train_upsampled["label"], random_state = 0)

In [16]:
model = pipeline_sgd.fit(x_train, y_train)
y_predict = model.predict(x_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9698546085100707